# Notebook to analysis authorship of random tweets

In [26]:
import os 
import sys
sys.path.append(os.path.abspath('..'))

In [27]:
import pickle
import pandas as pd
import credentials



In [ ]:
# load data from pickle files

with open('./data/twitter_train.pkl', 'rb') as f:
    twitter_train = pickle.load(f)


In [28]:
# Create the two large concatenated train sets

marker = "\n---TWEET---\n"
author_texts = {}

for author in twitter_train['author'].unique():
    tweets = twitter_train[twitter_train['author'] == author]['content'].astype(str)
    author_texts[author] = marker.join(tweets)

In [15]:
# Check the first few tweets for each author
for author, text in list(author_texts.items())[:5]:
    print(f"Author: {author}")
    print(text[:200]) 
    print("\n")



Author: rihanna
“@RihrihFreak: I knew Rih would RT that..didn't she get slap for doing that to her late gran? Lol” first thing that popped in my head bruh
---TWEET---
Bitchez be like...."#nomakeup " http://t.co/6juBO


Author: katyperry
Amazon prime and slime or nah?
---TWEET---
@iammelissalynn happy birthday qewtie!!! 😘😘😘
---TWEET---
U also made Kitty Purry happy. “@essesfadaperry: @katyperry Ready! downloaded! I am now happy and ro




In [16]:
# Check the number of tokens in the training set
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

for author, text in author_texts.items():
    tokens = encoding.encode(text)
    print(f"Author: {author}, Number of tokens: {len(tokens)}")


Author: rihanna, Number of tokens: 69587
Author: katyperry, Number of tokens: 75595


In [29]:
# Check the number of tweets in the training dataset
tweet_counts = twitter_train['author'].value_counts()
print(tweet_counts)


author
katyperry    2339
rihanna      2301
Name: count, dtype: int64


In [30]:
# Create a sample of the training dataset to use in the prompt

import numpy as np

marker = "\n---TWEET---\n"
sampled_author_texts = {}

for author in twitter_train['author'].unique():
    author_tweets = twitter_train[twitter_train['author'] == author]['content'].astype(str)
    n_total = len(author_tweets)
    n_sample = max(1, int(np.ceil(n_total / 3)))  # at least 1 tweet
    sampled_tweets = author_tweets.sample(n=n_sample, random_state=42)  # set random_state for reproducibility
    sampled_author_texts[author] = marker.join(sampled_tweets)


In [31]:
# Check the number of tokens in the sampled dataset
for author, text in sampled_author_texts.items():
    tokens = encoding.encode(text)
    print(f"Author: {author}, Number of tokens: {len(tokens)}")


Author: rihanna, Number of tokens: 23499
Author: katyperry, Number of tokens: 24853


In [19]:
# Get a random tweet from the test set
random_tweet_text = twitter_test['content'].sample(n=1, random_state=42).values[0]
print(random_tweet_text)
print(f"Author: {twitter_test.loc[twitter_test['content'] == random_tweet_text, 'author'].values[0]}")

She. X The guys. #MiamiNights http://t.co/q5g4Ml6bnA
Author: rihanna


In [32]:
# To save
with open('./data/author_texts.pkl', 'wb') as f:
    pickle.dump(author_texts, f)

with open('./data/sampled_author_texts.pkl', 'wb') as f:
    pickle.dump(sampled_author_texts, f)

In [24]:
# Create the template prompt

prompt = f"""
You are an expert in linguistic analysis and authorship attribution. Your task is to analyze authorship--i.e. idiolectal--markers that the \
texts reveal about their authors.


You will be provided with:
1. Training samples (tweets) from 2 different authors, Rihanna and Katy Perry
2. A new text whose author you need to identify between those two authors

Guideline for analysis:
- Just use state of the art techniques to attribute authorship

Rihanna's tweets:
{sampled_author_texts['rihanna']}

Katy Perry's tweets:
{sampled_author_texts['katyperry']}

New text to analyze:
{random_tweet_text}

Just return the author's name.

"""

## Using Claude

In [21]:
# With Anthropic's Claude
from anthropic import Anthropic
API_KEY = credentials.get_credentials_claude()

client = Anthropic(api_key=API_KEY)

In [22]:
# Create a completion function that will be used to query the model
def get_completion(prompt: str, max_tokens=5000) -> str:
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        )
    return response.content[0].text

In [25]:
first_prediction = get_completion(prompt)
print(first_prediction)

RateLimitError: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'This request would exceed the rate limit for your organization (95ce3372-8390-44d0-a412-13c7e00df683) of 40,000 input tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}